In [2]:


class VAE_info(IDVAE):
    def compute_information(self, theta):
        # compute conditional p(x|theta) (likelihood)
        prob = torch.squeeze(self.decoder(theta))

        # Loop though output dimensions/items, save Fisher information matrix for each
        FIMs = []
        for i in range(prob.shape[0]):
            p = prob[i]
            # compute gradients of log p(x=1) and log p(x=0) to the latent variables 
            
            gradient0 = torch.autograd.grad(torch.log(1-p), theta, retain_graph=True, allow_unused=True)[0]
            gradient1 = torch.autograd.grad(torch.log(p), theta, retain_graph=True, allow_unused=True)[0]
            # Compute the outer product of the gradients with itself
            outer0 = torch.outer(gradient0, gradient0)
            outer1 = torch.outer(gradient1, gradient1)

            # compute the Fisher information matrix by calculating the expectation of this outer product 
            FIM = p * outer1 + (1-p) * outer0

            # append to list of FIMs
            FIMs.append(FIM.detach().numpy())

        return FIMs
    
class Encoder(pl.LightningModule):
    """
    Neural network used as encoder
    """
    def __init__(self,
                 nitems: int,
                 latent_dims: int,
                 layer_sizes: list[int]):
        """
        Initialisation
        :param latent_dims: number of latent dimensions of the model
        """
        super(Encoder, self).__init__()

        input_layer = nitems
    
        self.dense1 = nn.Linear(input_layer, layer_sizes[0])
        self.dense2 = nn.Linear(layer_sizes[0], layer_sizes[1])
        self.densem = nn.Linear(layer_sizes[1], latent_dims)
        self.denses = nn.Linear(layer_sizes[1], latent_dims)


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        A forward pass though the encoder network
        :param x: a tensor representing a batch of response data
        :param m: a mask representing which data is missing
        :return: a sample from the latent dimensions
        """

        # calculate s and mu based on encoder weights
        out = F.elu(self.dense1(x))
        out = F.elu(self.dense2(out))
        mu =  self.densem(out)
        log_sigma = self.denses(out)

        return mu, log_sigma

class Decoder(pl.LightningModule):
    """
    Neural network used as decoder
    """

    def __init__(self, nitems: int, latent_dims: int, layer_sizes: list[int]):
        """
        Initialisation
        :param latent_dims: the number of latent factors
        :param qm: IxD Q-matrix specifying which items i<I load on which dimensions d<D
        """
        super().__init__()

        self.dense1 = nn.Linear(latent_dims, layer_sizes[0])
        self.dense2 = nn.Linear(layer_sizes[0], layer_sizes[1])
        self.dense3 = nn.Linear(layer_sizes[1], nitems)

        
        

    def forward(self, x: torch.Tensor):
        out = F.elu(self.dense1(x))
        out = F.elu(self.dense2(out))
        out = F.elu(self.dense3(out))
        out = F.sigmoid(out)

        return out
    


NameError: name 'IDVAE' is not defined